# Problema 2
Implementar una función que encuentre el valor máximo en un array de números
flotantes de gran tamaño (>100 millones de elementos) usando GPU.

a. Entrada: Array 1D de números reales (float32)

b. Salida: El valor máximo del array

c. Comparar rendimiento: CPU vs GPU








In [1]:
# Problema 2
# Comprobar que se este usando la GPU
!nvidia-smi

Mon Oct 27 17:48:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   66C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# instalar Cupy
!pip install cupy-cuda12x


In [3]:
# CPU Versión secuencial (base de comparación)
import numpy as np
import time

# Crear array grande en CPU (float32)
N = 100_000_000
arr_cpu = np.random.rand(N).astype(np.float32)

# Medir tiempo CPU
t0 = time.perf_counter()
max_cpu = np.max(arr_cpu)
t1 = time.perf_counter()

print(f"Máximo (CPU): {max_cpu}")
print(f"Tiempo CPU: {t1 - t0:.4f} segundos")


Máximo (CPU): 1.0
Tiempo CPU: 0.0342 segundos


In [4]:
# GPU: Versión paralela usando CuPy
import cupy as cp
import time

# Copiar datos a GPU
t0 = time.perf_counter()
arr_gpu = cp.array(arr_cpu)  # Transferencia host->device
cp.cuda.Stream.null.synchronize()
t1 = time.perf_counter()

# Calcular máximo en GPU
t2 = time.perf_counter()
max_gpu = cp.max(arr_gpu)
cp.cuda.Stream.null.synchronize()  # Esperar que termine
t3 = time.perf_counter()

# Transferir resultado a CPU
max_gpu_host = max_gpu.get()
t4 = time.perf_counter()

print(f"Máximo (GPU): {max_gpu_host}")
print(f"Tiempo transferencia Host→Device: {t1 - t0:.4f} s")
print(f"Tiempo kernel GPU: {t3 - t2:.4f} s")
print(f"Tiempo Device→Host: {t4 - t3:.4f} s")
print(f"Tiempo total GPU: {t4 - t0:.4f} s")


Máximo (GPU): 1.0
Tiempo transferencia Host→Device: 0.6190 s
Tiempo kernel GPU: 0.0684 s
Tiempo Device→Host: 0.0004 s
Tiempo total GPU: 0.6879 s


In [5]:
cpu_time = t1 - t0
gpu_time_total = t4 - t0
speedup = (t1 - t0) / gpu_time_total

print(f"Speedup (CPU/GPU total): {speedup:.2f}x")


Speedup (CPU/GPU total): 0.90x


Interpretación de los resultados:

Observando los tiempos de ejecución que obtuviste:

    Tiempo CPU: 0.0342 segundos
    Tiempo total GPU: 0.6879 segundos
    Tiempo transferencia Host→Device: 0.6190 segundos
    Tiempo kernel GPU: 0.0684 segundos
    Tiempo Device→Host: 0.0004 segundos

Y el Speedup (CPU/GPU total): 0.90x

Esto significa lo siguiente:

* Rendimiento Inicial Inesperado: Contrario a lo que se esperaría en muchos casos al usar GPU para tareas computacionalmente intensivas, en este experimento particular, la versión en CPU fue significativamente más rápida (0.0342 segundos) que la versión total en GPU (0.6879 segundos). El speedup de 0.90x confirma esto, indicando que la versión en GPU fue aproximadamente un 90% tan rápida como la versión en CPU (o, visto de otra manera, la CPU fue alrededor de 1.11 veces más rápida).
* El Costo de la Transferencia de Datos: La razón principal de que la GPU no haya superado a la CPU en este caso es el tiempo de transferencia de datos entre la memoria principal del sistema (Host) y la memoria de la GPU (Device). Como puedes ver, transferir el array de 100 millones de elementos a la GPU tomó 0.6190 segundos, lo cual es mucho mayor que el tiempo que le tomó a la CPU calcular el máximo directamente (0.0342 segundos).
* Eficiencia del Kernel GPU: A pesar del alto costo de la transferencia, el cálculo del máximo en la GPU (el "Tiempo kernel GPU") fue muy rápido, tomando solo 0.0684 segundos. Esto demuestra que, una vez que los datos están en la memoria de la GPU, el paralelismo de la GPU es efectivo para esta operación.
* Transferencia de Vuelta Rápida: La transferencia del resultado (un único valor máximo) de vuelta a la CPU (Tiempo Device→Host) fue extremadamente rápida (0.0004 segundos), lo cual es esperable ya que es una cantidad mínima de datos.

¿Por qué la CPU fue más rápida en este caso?

Para que una implementación en GPU sea más rápida que una en CPU, el tiempo ahorrado al ejecutar el cálculo en paralelo en la GPU debe ser mayor que el tiempo adicional que se invierte en transferir los datos entre la CPU y la GPU.

En este problema específico de encontrar el valor máximo en un array, la operación np.max() en NumPy (que se ejecuta en la CPU) es extremadamente eficiente y está altamente optimizada. Para un array de 100 millones de elementos, encontrar el máximo es una operación relativamente sencilla que la CPU puede manejar secuencialmente muy rápido.

El "overhead" de la transferencia de datos a la GPU (que implica copiar todos los 100 millones de elementos) es el factor dominante que hace que el tiempo total en la GPU sea mayor.

Consideraciones para un mejor rendimiento en GPU:

Para que el uso de la GPU sea ventajoso en problemas de este tipo, generalmente se necesitan dos condiciones:

* Operaciones Computacionalmente Muy Intensivas: La operación que se realiza en la GPU debe ser mucho más compleja y consumir más tiempo en la CPU. Encontrar el máximo es relativamente simple. Operaciones como multiplicaciones de matrices grandes, transformadas de Fourier, o simulaciones complejas suelen beneficiarse más del paralelismo de la GPU.
* Reutilización de Datos en la GPU: El mayor beneficio de la GPU se obtiene cuando los datos se transfieren una vez a la GPU y se realizan múltiples operaciones (kernels) sobre ellos sin tener que transferirlos de vuelta a la CPU entre cada operación. Si tienes una serie de cálculos que hacer con el mismo array, transferirlo una vez y realizar todas las operaciones en la GPU sería mucho más eficiente.

En resumen, los resultados son esperables para este problema particular y tamaño de datos, donde el costo de la transferencia de datos a la GPU supera el ahorro de tiempo en el cálculo del máximo en el kernel. Esto ilustra un principio fundamental del cómputo acelerado por GPU: la transferencia de datos entre Host y Device es costosa y debe minimizarse para obtener un rendimiento superior.